
# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
from keras.models import load_model
from string import ascii_lowercase 
import numpy as np
np.random.seed(6)
from keras_self_attention import SeqSelfAttention

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)


# Training

This code block involves model training and saving

In [ ]:
import random
import string
import time
import re
import collections
import numpy as np
from keras.layers import *
from keras.models import Model, load_model
from keras import optimizers, backend as K
from keras_self_attention import SeqSelfAttention
from string import ascii_lowercase 
np.random.seed(6)
import time

In [11]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
#         self.hangman_url = self.determine_hangman_url()
#         self.access_token = access_token
#         self.session = session or requests.Session()
#         self.timeout = timeout
        self.guessed_letters = []
        self.train_dictionary= []
        self.test_dictionary = []
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.train_dictionary)).most_common()
        self.maxLen = max(len(w) for w in self.train_dictionary)
        self.encoderDict=dict(zip(ascii_lowercase, range(26)))
        self.encoderDict["_"]=26 #missing letters
        self.encoderDict["."] =27 #padding
        self.decoderDict=dict(zip(range(26),ascii_lowercase))
        self.numTries=6
        self.outputs=[]
        self.inputs1=[]
        self.inputs2=[]
        self.remainingLetters=[]
        self.model=self.createModel(self.maxLen)
        print(self.model.summary())
        print()
        
        self.correctGuesses=0
        self.totalGuesses=0
        self.numWins=0
        self.totalGames=0
        
        self.model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.Adam(6e-4))
        #self.train()
        self.current_dictionary = []
        
#     @staticmethod
#     def determine_hangman_url():
#         links = ['https://trexsim.com', 'https://sg.trexsim.com']

#         data = {link: 0 for link in links}

#         for link in links:

#             requests.get(link)

#             for i in range(10):
#                 s = time.time()
#                 requests.get(link)
#                 data[link] = time.time() - s

#         link = sorted(data.items(), key=lambda x: x[1])[0][0]
#         link += '/trexsim/hangman'
#         return link
    def createModel(self,n=30):
        #5e-4
        
        inputPath1=Input(shape=(n,28,), name='path1')
        inputPath2=Input(shape=(26,), name='path2')
        #     path1=Embedding(n,100, mask_zero = True)(inputPath1)
        #     p1=Embedding(30,100, mask_zero = True)(p1)
        path1=Bidirectional(LSTM(256,return_sequences=True))(inputPath1)
        path1=SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,attention_activation='relu')(path1)
        path1=TimeDistributed(Dense(64,activation='gelu', kernel_initializer='uniform'))(path1)
        path1=Bidirectional(LSTM(256))(path1)
        #path2=SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,attention_activation='sigmoid')(path2)
        path1=Dense(256,activation='gelu', kernel_initializer='uniform')(path1)

        path2=Dense(128,activation='gelu', kernel_initializer='uniform')(inputPath2)
        path2=Dense(128,activation='gelu', kernel_initializer='uniform')(path2)
        merged = Concatenate()([path2, path1])
        merged = Dense(256,activation='gelu', kernel_initializer='uniform')(merged)
#         merged = Dense(256,activation='tanh', kernel_initializer='uniform')(merged)
        merged = Dense(26,activation='softmax', kernel_initializer='uniform')(merged)
        return Model(inputs=[inputPath1,inputPath2], outputs=merged, name='model')
    
    
    def reset(self, word):
        self.numTries=6
        self.outputs=[]
        self.inputs1=[]
        self.inputs2=[]
        self.guessed_letters=[]
        self.remainingLetters=dict(collections.Counter(word))
        
        
    def encode(self,word):
        if(len(word)<self.maxLen):word=word+"".join(["."]*(self.maxLen-len(word)))
        encoded=np.zeros((1,self.maxLen,28))
        for i in range(self.maxLen):
            if(word[i]=="." or word[i] in self.guessed_letters):
                    encoded[0,i,self.encoderDict[word[i]]]=1
            else:encoded[0,i,26]=1
        return encoded    
    def play(self,word):
        self.reset(word)
        
        while sum(self.remainingLetters.values())>0 and self.numTries>0:
            i1=self.encode(word)
            i2=np.zeros((1,26))
            for x in self.guessed_letters:
                i2[0,self.encoderDict[x]]=1
            output = np.zeros((1,26))
            for k,v in self.remainingLetters.items():
                output[0,self.encoderDict[k]] = v
            output[0]/=output[0].sum() 
            #print(i1.shape,i2.shape,output.shape)
            if len(self.inputs1)==0:
                self.inputs1,self.inputs2,self.outputs=i1,i2,output
            else:
#                 print(i1.shape,i2.shape,output.shape)
#                 print(self.inputs1.shape,self.inputs2.shape,self.outputs.shape)
                self.inputs1=np.append(self.inputs1,i1,axis=0)
                self.inputs2=np.append(self.inputs2,i2,axis=0)
                self.outputs=np.append(self.outputs,output,axis=0)
                
            guesses = np.flip(np.argsort(self.model.predict([i1,i2],verbose=None)))[0]
            #print(guesses)
            for i in guesses.tolist():
                guess = self.decoderDict[i]
                if guess not in self.guessed_letters:
                    self.guessed_letters.append(guess)
                    break
#             print(word,guess,self.remainingLetters)
#             print()
#             print(output)
#             print()
            self.totalGuesses+=1
            if guess in self.remainingLetters and self.remainingLetters[guess]>0:
                self.remainingLetters[guess]=0
                self.correctGuesses+=1
            else: self.numTries-=1  
 
        if(sum(self.remainingLetters.values())==0):
            self.numWins+=1
        self.totalGames+=1        
        return(self.inputs1,self.inputs2,self.outputs)
    
            
    def train(self):
        
        wordBatchSize=250
        epochs=1
        xTrain1=0
        xTrain2=0
        yTrain=0
        batchNum=0
        totalBatches = (len(self.train_dictionary)//wordBatchSize+1)
        for e in range(epochs):
            count=0
            t1=time.time()
            for word in self.train_dictionary:
                count+=1 
                input1, input2, output = self.play(word)
                if(count==1):
                    xTrain1=input1
                    xTrain2=input2
                    yTrain=output
                else:
#                     print(xTrain1.shape,xTrain2.shape,yTrain.shape)
#                     print(input1.shape,input2.shape,output.shape)
                    xTrain1=np.append(xTrain1,input1,axis=0)
                    xTrain2=np.append(xTrain2,input2,axis=0)
                    yTrain=np.append(yTrain,output,axis=0)
                if(count==wordBatchSize):
                    batchNum+=1
                    count=0
                    ##train function
                    print("Starting training for Batch: "+str(batchNum) +" /"+str(totalBatches)+" and Epoch: "+ str(e+1))
                    print(xTrain1.shape,xTrain2.shape,yTrain.shape)
                    self.model.fit([xTrain1,xTrain2],yTrain,batch_size=256)
                    print("Time taken in seconds to train the batch: "+str(time.time()-t1))
                    print("Correct Guesses: "+str(self.correctGuesses)+"/"+str(self.totalGuesses)+" , number of wins: "+
                         str(self.numWins)+"/"+str(self.totalGames))
                    t1=time.time()
                    
                    self.correctGuesses=0
                    self.totalGuesses=0
                    self.numWins=0
                    self.totalGames=0
                    xTrain1=0
                    xTrain2=0
                    yTrain=0
            count=0
            ##train function
            print("Completing training for Epoch: "+str(e+1))
#             self.model.fit([xTrain1,xTrain2],yTrain,batch_size=256)
#             xTrain1=0
#             xTrain2=0
#             yTrain=0
                    
    def trainOnTest(self):
        
        wordBatchSize=250
        epochs=1
        xTrain1=0
        xTrain2=0
        yTrain=0
        batchNum=0
        totalBatches = (len(self.test_dictionary)//wordBatchSize+1)
        for e in range(epochs):
            count=0
            t1=time.time()
            for word in self.test_dictionary:
                count+=1
                input1, input2, output = self.play(word)
                if(count==1):
                    xTrain1=input1
                    xTrain2=input2
                    yTrain=output
                else:
#                     print(xTrain1.shape,xTrain2.shape,yTrain.shape)
#                     print(input1.shape,input2.shape,output.shape)
                    xTrain1=np.append(xTrain1,input1,axis=0)
                    xTrain2=np.append(xTrain2,input2,axis=0)
                    yTrain=np.append(yTrain,output,axis=0)
                if(count==wordBatchSize):
                    batchNum+=1
                    count=0
                    ##train function
                    print("Starting training for Batch: "+str(batchNum) +" /"+str(totalBatches)+" and Epoch: "+ str(e+1))
                    print(xTrain1.shape,xTrain2.shape,yTrain.shape)
                    self.model.fit([xTrain1,xTrain2],yTrain,batch_size=256)
                    print("Time taken in seconds to train the batch: "+str(time.time()-t1))
                    print("Correct Guesses: "+str(self.correctGuesses)+"/"+str(self.totalGuesses)+" , number of wins: "+
                         str(self.numWins)+"/"+str(self.totalGames))
                    t1=time.time()
                    
                    self.correctGuesses=0
                    self.totalGuesses=0
                    self.numWins=0
                    self.totalGames=0
                    xTrain1=0
                    xTrain2=0
                    yTrain=0
            count=0
            ##train function
            print("Completing training for Epoch: "+str(e+1))
#             self.model.fit([xTrain1,xTrain2],yTrain,batch_size=256)
#             xTrain1=0
#             xTrain2=0
#             yTrain=0
    def loadModel(self,modelPath):
        self.model=load_model(modelPath)
    def test(self):
        t1=time.time()
        self.correctGuesses=0
        self.totalGuesses=0
        self.numWins=0
        self.totalGames=0
        for word in self.test_dictionary:
            self.play(word)
        print("Time taken in seconds to test the batch of "+str(len(self.test_dictionary)) +" words: "+str(time.time()-t1))
        print("Correct Guesses: "+str(self.correctGuesses)+"/"+str(self.totalGuesses)+" , number of wins: "+
             str(self.numWins)+"/"+str(self.totalGames))
        self.correctGuesses=0
        self.totalGuesses=0
        self.numWins=0
        self.totalGames=0



#             self.model.fit([xTrain1,xTrain2],yTrain,batch_size=256)
#             xTrain1=0
#             xTrain2=0
#             yTrain=0
                    
                                 
    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        word="".join(word)
        clean_word = word[::2].replace("_",".")
        
        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)
        
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()                   
        
        guess_letter = '!'
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break
            
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break            
        
        return guess_letter
    def match(self,word1,word2):
        word1=list(word1)
        word2=list(word2)
        if(len(word1)==len(word2)):
            for x in range(len(word1)):
                if word2[x]!="_" and word2[x]!=word1[x]:
                    return False
            return True
        else: return False    
    def guess1(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        word="".join(word)    
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
                
            # if dictionary word is a possible match then add it to the current dictionary
            if self.match(dict_word,word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)
        
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()                   
        
        guess_letter = '!'
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break
            
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break            
        
        return guess_letter

    def guess2(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        word="".join(word)
                
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
                
            # if dictionary word is a possible match then add it to the current dictionary
            if self.match(dict_word,word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        guess_letter = '!'
        d={}
        for x in range(len(word)):
            if word[x]=="_":
                sortedCount=collections.Counter("".join([a[x] for a in new_dictionary]))
                total = sum(sortedCount.values())
                sortedCount = {key:sortedCount[key]/total for key in sortedCount.keys()}
                sortedCount = sorted(sortedCount.items(), key=lambda x:-x[1])
                #print(sortedCount)
                d[x]=sortedCount
        maxProbalities = {}
        maxProb = 0 
        if(len(d)>0):
            for a in d.keys():
                for letter,prob in d[a]:
                    if letter not in self.guessed_letters:
                        if maxProb<prob:
                            maxProb = prob
                            guess_letter = letter
                            break
                        
#         # count occurrence of all characters in possible word matches
#         full_dict_string = "".join(new_dictionary)
        
#         c = collections.Counter(full_dict_string)
#         sorted_letter_count = c.most_common()                   
        
#         guess_letter = '!'
        
#         # return most frequently occurring letter in all possible words that hasn't been guessed yet
#         for letter,instance_count in sorted_letter_count:
#             if letter not in self.guessed_letters:
#                 guess_letter = letter
#                 break
            
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break            
        
        return guess_letter
    
    def guess3(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        word="".join(word)
                
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
                
            # if dictionary word is a possible match then add it to the current dictionary
            if self.match(dict_word,word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        guess_letter = '!'
        d={}
        for x in range(len(word)):
            totalProb = {}
            if word[x]=="_":
                sortedCount=collections.Counter("".join([a[x] for a in new_dictionary]))
                total = sum(sortedCount.values())
                sortedCount = {key:sortedCount[key]/total for key in sortedCount.keys()}
#                 sortedCount = sorted(sortedCount.items(), key=lambda x:-x[1])
                #print(sortedCount)
                d[x]=sortedCount
                totalProb = {x: totalProb.get(x, 0) + sortedCount.get(x, 0)
                    for x in set(totalProb).union(sortedCount)}
            d[-1]=sorted(totalProb.items(), key=lambda x:-x[1])   
        maxProbalities = {}
        maxProb = 0 
        if(len(d)>0):
            if(-1 in d):
                for letter,prob in d[-1]:
                    if letter not in self.guessed_letters:
                        if maxProb<prob:
                            maxProb = prob
                            guess_letter = letter
                            break
                
#             for a in d.keys():
#                 for letter,prob in d[a]:
#                     if letter not in self.guessed_letters:
#                         if maxProb<prob:
#                             maxProb = prob
#                             guess_letter = letter
#                             break
                        
#         # count occurrence of all characters in possible word matches
#         full_dict_string = "".join(new_dictionary)
        
#         c = collections.Counter(full_dict_string)
#         sorted_letter_count = c.most_common()                   
        
#         guess_letter = '!'
        
#         # return most frequently occurring letter in all possible words that hasn't been guessed yet
#         for letter,instance_count in sorted_letter_count:
#             if letter not in self.guessed_letters:
#                 guess_letter = letter
#                 break
            
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break            
        
        return guess_letter
    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = np.array(text_file.read().splitlines())
        np.random.shuffle(full_dictionary)
        numTrain = 210000
        self.train_dictionary= full_dictionary[:numTrain].tolist()
        self.test_dictionary = full_dictionary[numTrain:].tolist()
        text_file.close()
        return full_dictionary.tolist()
                
    def playGame(self, practice=True, verbose=True):
        
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        game_id=0
        successfulGames=0
        for word in self.test_dictionary:
            game_id+=1
            word = list(word)
            self.guessed_letters = []
            self.current_dictionary = self.train_dictionary
            tries_remains = 6
            p=1
            mask=np.random.choice(np.arange(2),len(word),p=[p,1-p])
            modifiedWord = [word[i] if mask[i]==1 else '_' for i in list(range(len(word)))]
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, "".join(modifiedWord)))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess3(modifiedWord)
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                a = [1 if (modifiedWord[i]=="_" and word[i]==guess_letter) else 0 for i in list(range(len(word)))]
                if(sum(a) == 0):tries_remains-=1
                else:
                    for x in range(len(word)):
                        if(a[x]==1):
                            modifiedWord[x] = word[x]
                status=""
                if(modifiedWord==word):status="success"
                elif(tries_remains==0):status="failed"
                else:status="ongoing"
                if status=="success":
                    successfulGames+=1
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    break
                elif status=="failed":
                    reason = "# of tries exceeded!"
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    break
                elif status=="ongoing":
                    if verbose:
                        print("Continuing game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, modifiedWord))
            
        print("Game finished with accuracy: {0}.".format(successfulGames/len(self.test_dictionary)))

In [4]:
api=HangmanAPI()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 path1 (InputLayer)             [(None, 29, 28)]     0           []                               
                                                                                                  
 bidirectional (Bidirectional)  (None, 29, 512)      583680      ['path1[0][0]']                  
                                                                                                  
 seq_self_attention (SeqSelfAtt  (None, 29, 512)     262145      ['bidirectional[0][0]']          
 ention)                                                                                          
                                                                                                  
 path2 (InputLayer)             [(None, 26)]         0           []                           

In [6]:
api.train()

Starting training for Batch: 1 /841 and Epoch: 1
(2724, 29, 28) (2724, 26) (2724, 26)
11/11 [==============================] - 9s 590ms/step - loss: 2.4835
Time taken in seconds to train the batch: 98.99405217170715
Correct Guesses: 1601/2724 , number of wins: 134/250
Starting training for Batch: 2 /841 and Epoch: 1
(2792, 29, 28) (2792, 26) (2792, 26)
11/11 [==============================] - 6s 550ms/step - loss: 2.4943
Time taken in seconds to train the batch: 93.72159504890442
Correct Guesses: 1629/2792 , number of wins: 125/250
Starting training for Batch: 3 /841 and Epoch: 1
(2792, 29, 28) (2792, 26) (2792, 26)
11/11 [==============================] - 6s 561ms/step - loss: 2.4798
Time taken in seconds to train the batch: 93.08905053138733
Correct Guesses: 1645/2792 , number of wins: 128/250
Starting training for Batch: 4 /841 and Epoch: 1
(2745, 29, 28) (2745, 26) (2745, 26)
11/11 [==============================] - 6s 553ms/step - loss: 2.5087
Time taken in seconds to train the ba

Starting training for Batch: 62 /841 and Epoch: 1
(2673, 29, 28) (2673, 26) (2673, 26)
11/11 [==============================] - 8s 719ms/step - loss: 2.5083
Time taken in seconds to train the batch: 97.84319543838501
Correct Guesses: 1508/2673 , number of wins: 116/250
Starting training for Batch: 63 /841 and Epoch: 1
(2768, 29, 28) (2768, 26) (2768, 26)
11/11 [==============================] - 8s 716ms/step - loss: 2.5036
Time taken in seconds to train the batch: 96.40946769714355
Correct Guesses: 1619/2768 , number of wins: 121/250
Starting training for Batch: 64 /841 and Epoch: 1
(2815, 29, 28) (2815, 26) (2815, 26)
11/11 [==============================] - 9s 809ms/step - loss: 2.4860
Time taken in seconds to train the batch: 101.66428852081299
Correct Guesses: 1672/2815 , number of wins: 128/250
Starting training for Batch: 65 /841 and Epoch: 1
(2776, 29, 28) (2776, 26) (2776, 26)
11/11 [==============================] - 9s 801ms/step - loss: 2.4835
Time taken in seconds to train t

11/11 [==============================] - 11s 1s/step - loss: 2.4940
Time taken in seconds to train the batch: 108.16635632514954
Correct Guesses: 1612/2730 , number of wins: 133/250
Starting training for Batch: 93 /841 and Epoch: 1
(2749, 29, 28) (2749, 26) (2749, 26)
11/11 [==============================] - 11s 1s/step - loss: 2.5044
Time taken in seconds to train the batch: 109.84275126457214
Correct Guesses: 1602/2749 , number of wins: 129/250
Starting training for Batch: 94 /841 and Epoch: 1
(2774, 29, 28) (2774, 26) (2774, 26)
11/11 [==============================] - 11s 998ms/step - loss: 2.5139
Time taken in seconds to train the batch: 109.90742087364197
Correct Guesses: 1598/2774 , number of wins: 125/250
Starting training for Batch: 95 /841 and Epoch: 1
(2701, 29, 28) (2701, 26) (2701, 26)
11/11 [==============================] - 11s 1s/step - loss: 2.5050
Time taken in seconds to train the batch: 107.82432174682617
Correct Guesses: 1539/2701 , number of wins: 123/250
Starting

Starting training for Batch: 123 /841 and Epoch: 1
(2729, 29, 28) (2729, 26) (2729, 26)
11/11 [==============================] - 13s 1s/step - loss: 2.4743
Time taken in seconds to train the batch: 101.89272809028625
Correct Guesses: 1614/2729 , number of wins: 135/250
Starting training for Batch: 124 /841 and Epoch: 1
(2739, 29, 28) (2739, 26) (2739, 26)
11/11 [==============================] - 16s 1s/step - loss: 2.5132
Time taken in seconds to train the batch: 105.21738195419312
Correct Guesses: 1544/2739 , number of wins: 118/250
Starting training for Batch: 125 /841 and Epoch: 1
(2795, 29, 28) (2795, 26) (2795, 26)
11/11 [==============================] - 13s 1s/step - loss: 2.5015
Time taken in seconds to train the batch: 120.38543939590454
Correct Guesses: 1633/2795 , number of wins: 124/250
Starting training for Batch: 126 /841 and Epoch: 1
(2746, 29, 28) (2746, 26) (2746, 26)
11/11 [==============================] - 14s 1s/step - loss: 2.5118
Time taken in seconds to train the

Starting training for Batch: 184 /841 and Epoch: 1
(2788, 29, 28) (2788, 26) (2788, 26)
11/11 [==============================] - 20s 2s/step - loss: 2.4754
Time taken in seconds to train the batch: 122.42218613624573
Correct Guesses: 1661/2788 , number of wins: 138/250
Starting training for Batch: 185 /841 and Epoch: 1
(2758, 29, 28) (2758, 26) (2758, 26)
11/11 [==============================] - 19s 2s/step - loss: 2.4571
Time taken in seconds to train the batch: 113.20474767684937
Correct Guesses: 1661/2758 , number of wins: 138/250
Starting training for Batch: 186 /841 and Epoch: 1
(2761, 29, 28) (2761, 26) (2761, 26)
11/11 [==============================] - 19s 2s/step - loss: 2.4860
Time taken in seconds to train the batch: 111.17377376556396
Correct Guesses: 1624/2761 , number of wins: 126/250
Starting training for Batch: 187 /841 and Epoch: 1
(2778, 29, 28) (2778, 26) (2778, 26)
11/11 [==============================] - 19s 2s/step - loss: 2.4900
Time taken in seconds to train the

...

Starting training for Batch: 611 /841 and Epoch: 1
(2693, 29, 28) (2693, 26) (2693, 26)
11/11 [==============================] - 87s 8s/step - loss: 2.4876
Time taken in seconds to train the batch: 197.77353882789612
Correct Guesses: 1579/2693 , number of wins: 136/250
Starting training for Batch: 612 /841 and Epoch: 1
(2782, 29, 28) (2782, 26) (2782, 26)
11/11 [==============================] - 87s 8s/step - loss: 2.4970
Time taken in seconds to train the batch: 201.7601556777954
Correct Guesses: 1622/2782 , number of wins: 123/250
Starting training for Batch: 613 /841 and Epoch: 1
(2804, 29, 28) (2804, 26) (2804, 26)
11/11 [==============================] - 88s 8s/step - loss: 2.4324
Time taken in seconds to train the batch: 204.06485557556152
Correct Guesses: 1700/2804 , number of wins: 140/250
Starting training for Batch: 614 /841 and Epoch: 1
(2752, 29, 28) (2752, 26) (2752, 26)
11/11 [==============================] - 90s 8s/step - loss: 2.5006
Time taken in seconds to train the 

Starting training for Batch: 672 /841 and Epoch: 1
(2785, 29, 28) (2785, 26) (2785, 26)
11/11 [==============================] - 101s 9s/step - loss: 2.4538
Time taken in seconds to train the batch: 221.93362545967102
Correct Guesses: 1691/2785 , number of wins: 136/250
Starting training for Batch: 673 /841 and Epoch: 1
(2752, 29, 28) (2752, 26) (2752, 26)
11/11 [==============================] - 104s 10s/step - loss: 2.4528
Time taken in seconds to train the batch: 224.1802065372467
Correct Guesses: 1649/2752 , number of wins: 140/250
Starting training for Batch: 674 /841 and Epoch: 1
(2781, 29, 28) (2781, 26) (2781, 26)
11/11 [==============================] - 102s 9s/step - loss: 2.4468
Time taken in seconds to train the batch: 223.33566331863403
Correct Guesses: 1716/2781 , number of wins: 148/250
Starting training for Batch: 675 /841 and Epoch: 1
(2719, 29, 28) (2719, 26) (2719, 26)
11/11 [==============================] - 105s 10s/step - loss: 2.5201
Time taken in seconds to trai

11/11 [==============================] - 109s 10s/step - loss: 2.4627
Time taken in seconds to train the batch: 227.77868700027466
Correct Guesses: 1580/2677 , number of wins: 132/250
Starting training for Batch: 703 /841 and Epoch: 1
(2719, 29, 28) (2719, 26) (2719, 26)
11/11 [==============================] - 111s 10s/step - loss: 2.4953
Time taken in seconds to train the batch: 232.13282132148743
Correct Guesses: 1593/2719 , number of wins: 132/250
Starting training for Batch: 704 /841 and Epoch: 1
(2758, 29, 28) (2758, 26) (2758, 26)
11/11 [==============================] - 110s 10s/step - loss: 2.4463
Time taken in seconds to train the batch: 232.64268374443054
Correct Guesses: 1632/2758 , number of wins: 125/250
Starting training for Batch: 705 /841 and Epoch: 1
(2706, 29, 28) (2706, 26) (2706, 26)
11/11 [==============================] - 111s 10s/step - loss: 2.4865
Time taken in seconds to train the batch: 232.01932001113892
Correct Guesses: 1576/2706 , number of wins: 126/250


11/11 [==============================] - 145s 13s/step - loss: 2.4704
Time taken in seconds to train the batch: 285.95004749298096
Correct Guesses: 1609/2674 , number of wins: 139/250
Starting training for Batch: 763 /841 and Epoch: 1
(2693, 29, 28) (2693, 26) (2693, 26)
11/11 [==============================] - 148s 13s/step - loss: 2.5010
Time taken in seconds to train the batch: 290.5632905960083
Correct Guesses: 1561/2693 , number of wins: 126/250
Starting training for Batch: 764 /841 and Epoch: 1
(2710, 29, 28) (2710, 26) (2710, 26)
11/11 [==============================] - 149s 14s/step - loss: 2.4581
Time taken in seconds to train the batch: 293.0048086643219
Correct Guesses: 1639/2710 , number of wins: 143/250
Starting training for Batch: 765 /841 and Epoch: 1
(2733, 29, 28) (2733, 26) (2733, 26)
11/11 [==============================] - 152s 14s/step - loss: 2.4691
Time taken in seconds to train the batch: 297.48117852211
Correct Guesses: 1613/2733 , number of wins: 142/250
Start

11/11 [==============================] - 150s 14s/step - loss: 2.4605
Time taken in seconds to train the batch: 288.1236484050751
Correct Guesses: 1653/2745 , number of wins: 137/250
Starting training for Batch: 823 /841 and Epoch: 1
(2707, 29, 28) (2707, 26) (2707, 26)
11/11 [==============================] - 149s 14s/step - loss: 2.4683
Time taken in seconds to train the batch: 284.9310348033905
Correct Guesses: 1571/2707 , number of wins: 138/250
Starting training for Batch: 824 /841 and Epoch: 1
(2684, 29, 28) (2684, 26) (2684, 26)
11/11 [==============================] - 149s 14s/step - loss: 2.4523
Time taken in seconds to train the batch: 283.71108055114746
Correct Guesses: 1582/2684 , number of wins: 136/250
Starting training for Batch: 825 /841 and Epoch: 1
(2742, 29, 28) (2742, 26) (2742, 26)
11/11 [==============================] - 152s 14s/step - loss: 2.4660
Time taken in seconds to train the batch: 289.78846502304077
Correct Guesses: 1651/2742 , number of wins: 138/250
St

In [14]:
api.trainOnTest()

Starting training for Batch: 1 /70 and Epoch: 1
(2701, 29, 28) (2701, 26) (2701, 26)
11/11 [==============================] - 15s 1s/step - loss: 2.4438
Time taken in seconds to train the batch: 106.41956114768982
Correct Guesses: 1993/3260 , number of wins: 178/300
Starting training for Batch: 2 /70 and Epoch: 1
(2673, 29, 28) (2673, 26) (2673, 26)
11/11 [==============================] - 14s 1s/step - loss: 2.3981
Time taken in seconds to train the batch: 104.41200757026672
Correct Guesses: 1627/2673 , number of wins: 156/250
Starting training for Batch: 3 /70 and Epoch: 1
(2632, 29, 28) (2632, 26) (2632, 26)
11/11 [==============================] - 15s 1s/step - loss: 2.4174
Time taken in seconds to train the batch: 103.2557463645935
Correct Guesses: 1640/2632 , number of wins: 161/250
Starting training for Batch: 4 /70 and Epoch: 1
(2674, 29, 28) (2674, 26) (2674, 26)
11/11 [==============================] - 15s 1s/step - loss: 2.4462
Time taken in seconds to train the batch: 105.0

Starting training for Batch: 32 /70 and Epoch: 1
(2662, 29, 28) (2662, 26) (2662, 26)
11/11 [==============================] - 18s 2s/step - loss: 2.4136
Time taken in seconds to train the batch: 108.2213487625122
Correct Guesses: 1708/2662 , number of wins: 168/250
Starting training for Batch: 33 /70 and Epoch: 1
(2663, 29, 28) (2663, 26) (2663, 26)
11/11 [==============================] - 18s 2s/step - loss: 2.4473
Time taken in seconds to train the batch: 108.42590665817261
Correct Guesses: 1614/2663 , number of wins: 145/250
Starting training for Batch: 34 /70 and Epoch: 1
(2758, 29, 28) (2758, 26) (2758, 26)
11/11 [==============================] - 18s 2s/step - loss: 2.3778
Time taken in seconds to train the batch: 111.85562038421631
Correct Guesses: 1736/2758 , number of wins: 166/250
Starting training for Batch: 35 /70 and Epoch: 1
(2655, 29, 28) (2655, 26) (2655, 26)
11/11 [==============================] - 18s 2s/step - loss: 2.4059
Time taken in seconds to train the batch: 1

Starting training for Batch: 63 /70 and Epoch: 1
(2691, 29, 28) (2691, 26) (2691, 26)
11/11 [==============================] - 21s 2s/step - loss: 2.4374
Time taken in seconds to train the batch: 114.26749086380005
Correct Guesses: 1638/2691 , number of wins: 145/250
Starting training for Batch: 64 /70 and Epoch: 1
(2674, 29, 28) (2674, 26) (2674, 26)
11/11 [==============================] - 21s 2s/step - loss: 2.4537
Time taken in seconds to train the batch: 113.38641715049744
Correct Guesses: 1666/2674 , number of wins: 155/250
Starting training for Batch: 65 /70 and Epoch: 1
(2609, 29, 28) (2609, 26) (2609, 26)
11/11 [==============================] - 20s 2s/step - loss: 2.4627
Time taken in seconds to train the batch: 110.39748811721802
Correct Guesses: 1576/2609 , number of wins: 147/250
Starting training for Batch: 66 /70 and Epoch: 1
(2703, 29, 28) (2703, 26) (2703, 26)
11/11 [==============================] - 22s 2s/step - loss: 2.4045
Time taken in seconds to train the batch: 

In [ ]:
api.model.save("model3_3.h5")

# Final Submission

In [2]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.model=load_model("model3_3.h5",custom_objects={"SeqSelfAttention":SeqSelfAttention})
        self.maxLen = 29
        self.encoderDict=dict(zip(ascii_lowercase, range(26)))
        self.encoderDict["_"]=26 #missing letters
        self.encoderDict["."] =27 #padding
        self.decoderDict=dict(zip(range(26),ascii_lowercase))
        self.current_dictionary = []
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link
        
    def encode(self,word):
        
        if(len(word)<self.maxLen):word=word+"".join(["."]*(self.maxLen-len(word)))
        encoded=np.zeros((1,self.maxLen,28))
        for i in range(self.maxLen):
            if(word[i]=="." or word[i] in self.guessed_letters):
                    encoded[0,i,self.encoderDict[word[i]]]=1
            else:encoded[0,i,26]=1
        return encoded    
    def guess(self, word):
        word = word[::2]
        i1=self.encode(word)
        i2=np.zeros((1,26))
        for x in self.guessed_letters:
            i2[0,self.encoderDict[x]]=1
        guesses = np.flip(np.argsort(self.model.predict([i1,i2],verbose=None)))[0]
        guess_letter='!'
        for i in guesses.tolist():
            guess = self.decoderDict[i]
            if guess not in self.guessed_letters:
                guess_letter=guess
                break
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break 
        return guess_letter            
    #             print(word,guess,self.remainingLetters)
    #             print()
    #             print(output)
    #             print()
    def guess2(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".")
        
        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)
        
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()                   
        
        guess_letter = '!'
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break
            
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break            
        
        return guess_letter
    def guess3(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        
                
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
                
            # if dictionary word is a possible match then add it to the current dictionary
            if self.match(dict_word,word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        guess_letter = '!'
        d={}
        for x in range(len(word)):
            totalProb = {}
            if word[x]=="_":
                sortedCount=collections.Counter("".join([a[x] for a in new_dictionary]))
                total = sum(sortedCount.values())
                sortedCount = {key:sortedCount[key]/total for key in sortedCount.keys()}
#                 sortedCount = sorted(sortedCount.items(), key=lambda x:-x[1])
                #print(sortedCount)
                d[x]=sortedCount
                totalProb = {x: totalProb.get(x, 0) + sortedCount.get(x, 0)
                    for x in set(totalProb).union(sortedCount)}
            d[-1]=sorted(totalProb.items(), key=lambda x:-x[1])   
        maxProbalities = {}
        maxProb = 0 
        if(len(d)>0):
            if(-1 in d):
                for letter,prob in d[-1]:
                    if letter not in self.guessed_letters:
                        if maxProb<prob:
                            maxProb = prob
                            guess_letter = letter
                            break
                
#             for a in d.keys():
#                 for letter,prob in d[a]:
#                     if letter not in self.guessed_letters:
#                         if maxProb<prob:
#                             maxProb = prob
#                             guess_letter = letter
#                             break
                        
#         # count occurrence of all characters in possible word matches
#         full_dict_string = "".join(new_dictionary)
        
#         c = collections.Counter(full_dict_string)
#         sorted_letter_count = c.most_common()                   
        
#         guess_letter = '!'
        
#         # return most frequently occurring letter in all possible words that hasn't been guessed yet
#         for letter,instance_count in sorted_letter_count:
#             if letter not in self.guessed_letters:
#                 guess_letter = letter
#                 break
            
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break            
        
        return guess_letter
    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
    
    def match(self,word1,word2):
        word1=list(word1)
        word2=list(word2)
        if(len(word1)==len(word2)):
            for x in range(len(word1)):
                if word2[x]!="_" and word2[x]!=word1[x]:
                    return False
            return True
        else: return False                 
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [3]:
api = HangmanAPI(access_token="c66cf58acd4576d68a22e165b1f254", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [5]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: c91202669577. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: i
Sever response: {'game_id': 'c91202669577', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ i _ _ _ _ _ _ _ _ _ i _ _ '}
Guessing letter: r
Sever response: {'game_id': 'c91202669577', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ i _ _ _ _ _ _ _ _ r i _ _ '}
Guessing letter: t
Sever response: {'game_id': 'c91202669577', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ i _ _ t _ _ _ _ _ r i _ _ '}
Guessing letter: e
Sever response: {'game_id': 'c91202669577', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ e i _ _ t _ e _ _ _ r i _ _ '}
Guessing letter: o
Sever response: {'game_id': 'c91202669577', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e i _ _ t _ e _ _ _ r i _ _ '}
Guessing letter: a
Sever response: {'game_id': 'c91202669577', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e i _ _ t _ e a _ _ r i _ _ '}
Guessin

In [6]:
for x in range(600):
    api.start_game(practice=1,verbose=False)
    if(x%50==0):
        print("Played "+str(x)+" games")
    time.sleep(0.5)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))
    

Played 0 games
Played 50 games
Played 100 games
Played 150 games
Played 200 games
Played 250 games
Played 300 games
Played 350 games
Played 400 games
Played 450 games
Played 500 games
Played 550 games
run 14600 practice games out of an allotted 100,000. practice success rate so far = 0.481


## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [7]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Playing  1  th game
Playing  2  th game
Playing  3  th game
Playing  4  th game
Playing  5  th game
Playing  6  th game
Playing  7  th game
Playing  8  th game
Playing  9  th game
Playing  10  th game
Playing  11  th game
Playing  12  th game
Playing  13  th game
Playing  14  th game
Playing  15  th game
Playing  16  th game
Playing  17  th game
Playing  18  th game
Playing  19  th game
Playing  20  th game
Playing  21  th game
Playing  22  th game
Playing  23  th game
Playing  24  th game
Playing  25  th game
Playing  26  th game
Playing  27  th game
Playing  28  th game
Playing  29  th game
Playing  30  th game
Playing  31  th game
Playing  32  th game
Playing  33  th game
Playing  34  th game
Playing  35  th game
Playing  36  th game
Playing  37  th game
Playing  38  th game
Playing  39  th game
Playing  40  th game
Playing  41  th game
Playing  42  th game
Playing  43  th game
Playing  44  th game
Playing  45  th game
Playing  46  th game
Playing  47  th game
Pl

Playing  378  th game
Playing  379  th game
Playing  380  th game
Playing  381  th game
Playing  382  th game
Playing  383  th game
Playing  384  th game
Playing  385  th game
Playing  386  th game
Playing  387  th game
Playing  388  th game
Playing  389  th game
Playing  390  th game
Playing  391  th game
Playing  392  th game
Playing  393  th game
Playing  394  th game
Playing  395  th game
Playing  396  th game
Playing  397  th game
Playing  398  th game
Playing  399  th game
Playing  400  th game
Playing  401  th game
Playing  402  th game
Playing  403  th game
Playing  404  th game
Playing  405  th game
Playing  406  th game
Playing  407  th game
Playing  408  th game
Playing  409  th game
Playing  410  th game
Playing  411  th game
Playing  412  th game
Playing  413  th game
Playing  414  th game
Playing  415  th game
Playing  416  th game
Playing  417  th game
Playing  418  th game
Playing  419  th game
Playing  420  th game
Playing  421  th game
Playing  422  th game
Playing  4

Playing  751  th game
Playing  752  th game
Playing  753  th game
Playing  754  th game
Playing  755  th game
Playing  756  th game
Playing  757  th game
Playing  758  th game
Playing  759  th game
Playing  760  th game
Playing  761  th game
Playing  762  th game
Playing  763  th game
Playing  764  th game
Playing  765  th game
Playing  766  th game
Playing  767  th game
Playing  768  th game
Playing  769  th game
Playing  770  th game
Playing  771  th game
Playing  772  th game
Playing  773  th game
Playing  774  th game
Playing  775  th game
Playing  776  th game
Playing  777  th game
Playing  778  th game
Playing  779  th game
Playing  780  th game
Playing  781  th game
Playing  782  th game
Playing  783  th game
Playing  784  th game
Playing  785  th game
Playing  786  th game
Playing  787  th game
Playing  788  th game
Playing  789  th game
Playing  790  th game
Playing  791  th game
Playing  792  th game
Playing  793  th game
Playing  794  th game
Playing  795  th game
Playing  7

HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [9]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.517


In [8]:
api.my_status() 

[14602, 1000, 517, 7021]